In [11]:
# Import required libraries
import xarray as xr
from pathlib import Path
import numpy as np
import datetime
import pandas as pd
import cfgrib
import os
import shutil
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [12]:
# Read the original datacube
ds_original = xr.open_dataset(Path.home() / 'hdd1/skondylatos/uc3/greece_big_zipped.nc')

In [13]:
ds_original

<xarray.Dataset>
Dimensions:                  (time: 4560, x: 1253, y: 983)
Coordinates:
  * time                     (time) datetime64[ns] 2009-03-06T10:00:00 ... 20...
  * x                        (x) float64 18.7 18.71 18.72 ... 28.88 28.89 28.9
  * y                        (y) float64 42.3 42.29 42.28 ... 34.32 34.31 34.3
    band                     int64 ...
    spatial_ref              int64 ...
Data variables: (12/90)
    1 km 16 days NDVI        (time, y, x) float32 ...
    1 km 16 days EVI         (time, y, x) float32 ...
    ET_500m                  (time, y, x) float32 ...
    LST_Day_1km              (time, y, x) float32 ...
    LST_Night_1km            (time, y, x) float32 ...
    Fpar_500m                (time, y, x) float32 ...
    ...                       ...
    population_density_2016  (y, x) float32 ...
    population_density_2017  (y, x) float32 ...
    population_density_2018  (y, x) float32 ...
    population_density_2019  (y, x) float32 ...
    population_density_2020  (y, x) float32 ...
    population_density_2021  (y, x) float32 ...
Attributes: (12/37)
    CDI:                        Climate Data Interface version 1.9.9rc1 (http...
    Conventions:                CF-1.6
    history:                    Mon Feb 07 16:26:09 2022: cdo mergetime greec...
    Source_Software:            dbinterface.py, dbexport.py, netcdf_handling.py
    creator_name:               Carolina Arias Munoz
    _CoordSysBuilder:           ucar.nc2.dataset.conv.CF1Convention
    ...                         ...
    License:                    Copernicus
    version:                    3.1
    NCO:                        netCDF Operators version 4.7.8 (Homepage = ht...
    cdo_openmp_thread_number:   8
    CDO:                        Climate Data Operators version 1.9.9rc1 (http...
    crs:                        EPSG:4326

In [14]:
def create_new_ds(ds):
    grid_x, grid_y = np.meshgrid(ds_original['x'], ds_original['y'])
    coords = np.stack([ds['lon'].values, ds['lat'].values], axis=2)
    coords = coords.reshape(166*198,2)
    
    t2m = ds['t_2m'].squeeze().values.reshape(166*198)
    gd_t2m = griddata(coords, t2m, (grid_x, grid_y), method='nearest')
    
    relhum_2m = ds['relhum_2m'].squeeze().values.reshape(166*198)
    gd_relhum_2m = griddata(coords, relhum_2m, (grid_x, grid_y), method='nearest')
    
    w10u = ds['10u'].sel(height = 10.0).squeeze().values.reshape(166*198)
    gd_w10u = griddata(coords, w10u, (grid_x, grid_y), method='nearest')
    
    w10v = ds['10v'].sel(height = 10.0).squeeze().values.reshape(166*198)
    gd_w10v = griddata(coords, w10v, (grid_x, grid_y), method='nearest')
    
    apcp = ds['apcp'].squeeze().values.reshape(166*198)
    gd_apcp = griddata(coords, apcp, (grid_x, grid_y), method='nearest')
    
    ds1 = xr.Dataset(
        data_vars=dict(
            t2m=(["y", "x"], gd_t2m),
            relhum_2m=(["y", "x"], gd_relhum_2m),
            w10u=(["y", "x"], gd_w10u),
            w10v=(["y", "x"], gd_w10v),
            apcp=(["y", "x"], gd_apcp)
        ),
        coords=dict(
            x=ds_original['x'].values,
            y=ds_original['y'].values
        )
    )
    
    ds1['apcp'] = ds1['apcp'] / 1000
    ds1['d_2m'] = (243.04 * ( np.log(ds1['relhum_2m']/100) + ( ( 17.625*(ds1['t2m'] -273.15) ) / (243.04+ds1['t2m']-273.15) ) ) / (17.625 - np.log(ds1['relhum_2m']/100) - ( (17.625* (ds1['t2m']-273.15) ) / (243.04+ds1['t2m']-273.15)))) + 273.15
    ds1['relhum_2m'] = ds1['relhum_2m'] / 100
    ds1['wind_speed'] = np.sqrt(ds1['w10u'] ** 2 + ds1['w10v'] ** 2)
    
    ds1 = ds1.expand_dims(time=ds.time)
    return ds1

In [15]:
months_31days = [5, 7, 8, 10]
daily = []
for month in range(5,11):
    for day in range(1,32):
        dses = []
        if(month < 10):
            if month in months_31days:
                if day < 10:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + '0' + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + '0' + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
                else:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
            else:
                if day < 10:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + '0' + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + '0' + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
                elif day < 31:
                    if day == 15 and month == 9:
                        for hour in range(12):
                            if hour < 10:
                                path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '12_00' + str(hour) + '.nc'
                                path = Path.home() / path
                            else:
                                path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '12_0' + str(hour) + '.nc'
                                path = Path.home() / path
                            tmp = xr.open_dataset(path)
                            dses.append(create_new_ds(tmp))
                        continue
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_20210' + str(month) + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))        
        else:
            if month in months_31days:
                if day < 10:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + '0' + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + '0' + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
                else:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
            else:
                if day < 10:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + '0' + str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) + '0' + str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
                elif day < 31:
                    for hour in range(24):
                        if hour < 10:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) +  str(day) + '00_00' + str(hour) + '.nc'
                            path = Path.home() / path
                        else:
                            path = 'jh-shared/METEO_DATA/PAPOU_DATA/BOLAM_DATA/BOL-H_2021' + str(month) +  str(day) + '00_0' + str(hour) + '.nc'
                            path = Path.home() / path
                        tmp = xr.open_dataset(path)
                        dses.append(create_new_ds(tmp))
        if len(dses) > 0:
            x = xr.merge(dses, compat='no_conflicts')

            time = dses[0].time.values

            max_ds = xr.Dataset(
                data_vars=dict(
                    max_t2m=(["y", "x"], x['t2m'].max(dim = 'time').data),
                    min_relhum_2m=(["y", "x"], x['relhum_2m'].min(dim = 'time').data),
                    max_d2m=(["y", "x"], x['d_2m'].max(dim = 'time').data),
                    max_wind_speed=(["y", "x"], x['wind_speed'].max(dim = 'time').data),
                    max_apcp=(["y", "x"], x['apcp'].sum(dim = 'time').data)
                ),
                coords=dict(
                    x=ds_original['x'].values,
                    y=ds_original['y'].values
                )
            )
            max_ds = max_ds.expand_dims(time=time)
            daily.append(max_ds)
x = xr.merge([daily[0], daily[1]])
for i in range(2,len(daily)):
    x = xr.merge([x, daily[i]])

In [19]:
path = Path.home() / 'hdd1/diogenis/observatory/bolam_meteo.nc'
x.to_netcdf(path)